In [11]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import wrangle as w
import wrangle_test as wt
import explore as e

import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from sklearn import metrics
from imblearn.over_sampling import SMOTENC

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

In [12]:
df = w.get_cws_data()

In [13]:
y_train, y_val, y_test, train_scaled, val_scaled, test_scaled,train, validate, test, df = wt.model_prep(df)

In [14]:
def upsample(train):
    '''takes a train dataframe, upsamples minority and returns upsampled X_train and upsampled y_train'''
    
    
    df_majority = train[train.bad_resident ==0]
    df_minority = train[train.bad_resident ==1]
    df_minority_upsampled = resample(df_minority, replace=True, n_samples=300 , random_state=91)
    
    # get upsampled_df
    df_upsampled = pd.concat([df_minority_upsampled, df_majority])
    
    # scale df_upsampled
    df_upsampled[['rent', 'monthly_inc', 'total_inc','age', 'risk_score']] = StandardScaler().fit_transform(df_upsampled[['rent', 'monthly_inc', 'total_inc','age', 'risk_score']])
    
    # split scaled df_upsampled
    X_train_upsampled = df_upsampled.drop('bad_resident',axis=1)
    y_train_upsampled = df_upsampled['bad_resident']
    
    return X_train_upsampled, y_train_upsampled

In [15]:
def get_knn(train, X_validate, y_validate):
    ''' takes a train dataframe, X_ validate, y_validate, print KNN  confusion matrix and classifaction report on train and validate data'''
    
    # get upsampled X_train and Y_train
    X_train_upsampled, y_train_upsampled = upsample(train)
    
    # create model
    knn= KNeighborsClassifier(n_neighbors =3)

    # fit the model to train data
    knn.fit(X_train_upsampled, y_train_upsampled)
    
    # make prediction on train obeservations
    y_pred = knn.predict(X_train_upsampled)
    
    # make prediction on validate obeservations
    y_pred_val = knn.predict(X_validate)
    
    # get confusion matrix
    confusion_matrix_train = metrics.confusion_matrix(y_train_upsampled, y_pred)
    confusion_matrix_val = metrics.confusion_matrix(y_validate, y_pred_val)
    
    # print confusion matrix and classification report
#     print('Confusion Matrix of upsampled Train:')
#     print(confusion_matrix_train)
#     print('\n Classification Report on Train:')
#     print(classification_report(y_train_upsampled, y_pred))
#     print('-----------------------------------------------------------')
#     print('Confusion Matrix of Validate:')
#     print( confusion_matrix_val)
#     print('\n Classification Report on Validate: ')
#     print(classification_report(y_validate, y_pred_val))
#     print()
    
    df1 = pd.DataFrame(classification_report(y_train_upsampled, y_pred, labels=['0','1'], output_dict=True)).T
    df2 = pd.DataFrame(classification_report(y_validate, y_pred_val, labels=['0','1'], output_dict=True)).T
    
#     print(df1.iloc[0:2,0:2])
#     print()
#     print(df2.iloc[0:2,0:2])

    train_df = df1.iloc[0:2,1:2]
    validate_df = df2.iloc[0:2,1:2]
    
#     train_df.rename(columns = {'precision':'precision_train_KNN', 'recall': 'recall_train_KNN'}, inplace=True)
#     validate_df.rename(columns = {'precision':'precision_validate_KNN', 'recall': 'recall_validate_KNN'}, inplace=True)

    train_df.rename(columns = {'recall': 'recall_train_KNN'}, inplace=True)
    validate_df.rename(columns = {'recall': 'recall_validate_KNN'}, inplace=True)

    concated_df = pd.concat([train_df,validate_df],axis=1) 
    
    return concated_df

In [16]:
kk = get_knn(train, val_scaled, y_val)
kk

,recall_train_KNN,recall_validate_KNN
0,0.97474,0.953206
1,0.94000,0.130435
